In [1]:
#################################################
# IMDB ETL - Init
#################################################

import pandas as pd
import numpy  as np
###
# Null date conversion fun
def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

In [2]:
import pandas as pd
import numpy  as np

#################################################
# Principals
#################################################

P = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
P["cat_cln"] = np.where (P['category'].str.contains('actor') | P['category'].str.contains('actress')
                         ,'actor'
                         ,P['category'])
P["cat_cln"] = np.where (P['category'].str.contains('self') | P['category'].str.contains('archive_footage')
                         ,'self',P['cat_cln'])


P.head(5)

,tconst,ordering,nconst,category,job,characters,cat_cln
0,tt0000001,1,nm1588970,self,\N,"[""Self""]",self
1,tt0000001,2,nm0005690,director,\N,\N,director
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N,cinematographer
3,tt0000002,1,nm0721526,director,\N,\N,director
4,tt0000002,2,nm1335271,composer,\N,\N,composer


In [3]:
import pandas as pd
import numpy  as np
#################################################
# Titles
#################################################

def conv(val):
    if not val:
        return 0    
    try:
        return np.int(val)
    except:        
        return np.int(0)

T = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t'
                 , encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                #, error_bad_lines=False
                #, dtype={'startYear': int }
                 , converters={'startYear':conv,'endYear':conv}                
                ) 
T["flg_doc"]           = np.where (T['genres'].str.contains('Documentary'),'Documentary','Fiction')
T["primaryTitle_year"] = T['primaryTitle']  + ' ('+  T['startYear'].map(str)  + ')'
## filter movie 
T = T.loc[T.titleType == 'movie', :]
T.head(10)

C:\Users\cw13\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,0,45,Romance,Fiction,Miss Jerry (1894)
498,tt0000502,movie,Bohemios,Bohemios,0,1905,0,100,\N,Fiction,Bohemios (1905)
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,0,70,"Action,Adventure,Biography",Fiction,The Story of the Kelly Gang (1906)
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,0,90,Drama,Fiction,The Prodigal Son (1907)
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0,1907,0,\N,Drama,Fiction,Robbery Under Arms (1907)
625,tt0000630,movie,Hamlet,Amleto,0,1908,0,\N,Drama,Fiction,Hamlet (1908)
668,tt0000675,movie,Don Quijote,Don Quijote,0,1908,0,\N,Drama,Fiction,Don Quijote (1908)
672,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,0,120,"Adventure,Fantasy",Fiction,The Fairylogue and Radio-Plays (1908)
783,tt0000793,movie,Andreas Hofer,Andreas Hofer,0,1909,0,\N,Drama,Fiction,Andreas Hofer (1909)
804,tt0000814,movie,La bocana de Mar Chica,La bocana de Mar Chica,0,1909,0,\N,\N,Fiction,La bocana de Mar Chica (1909)


In [4]:
var_list = ('tconst','primaryTitle' , 'titleType')
#D = newdf.loc[:,('const','title', 'dat_per', 'title_type')]
D = T.loc[:, var_list]
D.head()
pd.get_dummies(D, columns=['titleType']).head(3)


,tconst,primaryTitle,titleType_movie
8,tt0000009,Miss Jerry,1
498,tt0000502,Bohemios,1
570,tt0000574,The Story of the Kelly Gang,1


In [5]:
#################################################
# Names
#################################################
    
N = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                 , converters={'birthYear':conv,'deathYear':conv}         
                )


#N["primaryName_year"] = N['primaryName']  + ' ('+  N['birthYear'].map(str) + ' - ' +  N['deathYear'].map(str)  + ')'

N["primaryName_year"] = N['primaryName']  + np.where(N['birthYear']==0, ' ',  ' ('+  N['birthYear'].map(str) + ' - '  +  np.where (N['deathYear'] == 0, ' ',  N['deathYear'].map(str)) + ')')
N.head(3)


,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles,primaryName_year
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0053137,tt0050419",Fred Astaire (1899 - 1987)
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0117057,tt0037382,tt0071877,tt0038355",Lauren Bacall (1924 - 2014)
2,nm0000003,Brigitte Bardot,1934,0,"actress,soundtrack,music_department","tt0056404,tt0049189,tt0057345,tt0054452",Brigitte Bardot (1934 - )


In [6]:
#################################################
# Ratings
#################################################
R = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz'
                 , compression='gzip'
                 , delimiter='\t',encoding='utf-8'
                 , header=0
                # , sep= " "
                 , quotechar='"'
                 #, error_bad_lines=False
                )
R.rename(columns = {  "averageRating": "tconst_imdb_rating"
                      , "numVotes":   "tconst_imdb_num_votes"
                      , "Const": "tconst"}
                      , inplace = True)
R.tail(3)

,tconst,tconst_imdb_rating,tconst_imdb_num_votes
1256804,tt9916730,8.4,6
1256805,tt9916766,6.8,21
1256806,tt9916778,7.2,35


In [7]:
############################################ 
### import my Ratings 
############################################ 
myR = pd.DataFrame()
myR = pd.read_csv ("data/ratings.csv", encoding='latin-1')
myR.rename(columns = {  "Your Rating": "tconst_your_rating"
                      , "Num Votes":   "tconst_your_num_votes"
                      , "Const": "tconst"}
                      , inplace = True)
myR.tail(3)

,tconst,tconst_your_rating,Date Rated,Title,URL,Title Type,IMDb Rating,Runtime (mins),Year,Genres,tconst_your_num_votes,Release Date,Directors
905,tt0989757,6,2015-10-11,Dear John,https://www.imdb.com/title/tt0989757/,movie,6.3,108.0,2010,"Drama, Mystery, Romance, Thriller, War",148877.0,2010-01-24,Lasse HallstrÃ¶m
906,tt0099699,7,2013-01-01,Green Card,https://www.imdb.com/title/tt0099699/,movie,6.3,107.0,1990,"Comedy, Drama, Romance",24565.0,1990-12-23,Peter Weir
907,tt0099703,6,2011-09-03,The Grifters,https://www.imdb.com/title/tt0099703/,movie,6.9,110.0,1990,"Crime, Drama, Thriller",28204.0,1990-09-14,Stephen Frears


In [8]:
#################################################
# Join
#################################################
from datetime import datetime
             
imdb = pd.merge(    P
                  , T 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(     N.loc[:, ('nconst','primaryProfession', 'primaryName', 'birthYear', 'deathYear', 'primaryName_year')]
                  , imdb
                  , how = 'inner', left_on="nconst", right_on="nconst"
                 )
imdb = pd.merge(     R.loc[:, ('tconst','tconst_imdb_rating', 'tconst_imdb_num_votes')]
                  , imdb
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
imdb = pd.merge(    myR.loc[:, ('tconst','tconst_your_rating')]
                  , imdb
                  , how = 'right', left_on="tconst", right_on="tconst"
                 )
imdb['rate_diff']           = imdb['tconst_your_rating'] - imdb['tconst_imdb_rating']
##imdb["age_crew"]            = int(imdb["birthYear_cln"]) -  int(imdb["startYear_cln"])
imdb["last_refresh"]        = datetime.now().strftime('%Y%m%d%H%M%S')
print(P.shape, imdb.shape)


(51750296, 7) (2505022, 27)


In [9]:
imdb[imdb['tconst'] == "tt1745960"].head(15)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,rate_diff,last_refresh
2565,tt1745960,7.0,8.5,296003,nm0000124,actress,Jennifer Connelly,1970,0,Jennifer Connelly (1970 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2566,tt1745960,7.0,8.5,296003,nm0000129,"actor,producer,soundtrack",Tom Cruise,1962,0,Tom Cruise (1962 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2567,tt1745960,7.0,8.5,296003,nm0000174,"actor,soundtrack,producer",Val Kilmer,1959,0,Val Kilmer (1959 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2568,tt1745960,7.0,8.5,296003,nm0143596,writer,Jim Cash,1941,2000,Jim Cash (1941 - 2000),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2569,tt1745960,7.0,8.5,296003,nm0185976,"writer,producer,director",Peter Craig,1969,0,Peter Craig (1969 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2570,tt1745960,7.0,8.5,296003,nm0258390,"writer,camera_department,producer",Jack Epps Jr.,1949,0,Jack Epps Jr. (1949 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2571,tt1745960,7.0,8.5,296003,nm0472567,"writer,producer,executive",Ehren Kruger,1972,0,Ehren Kruger (1972 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2572,tt1745960,7.0,8.5,296003,nm1098479,"writer,producer,miscellaneous",Justin Marks,0,0,Justin Marks,...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2573,tt1745960,7.0,8.5,296003,nm1886602,"actor,producer,soundtrack",Miles Teller,1987,0,Miles Teller (1987 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833
2574,tt1745960,7.0,8.5,296003,nm2676052,"director,writer,producer",Joseph Kosinski,1974,0,Joseph Kosinski (1974 - ),...,Top Gun: Maverick,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833


In [10]:
#################################################
# add row number for filmmakers with more than one release per year
#################################################
imdb['RN'] = imdb.sort_values(['nconst', 'tconst'], ascending=[True, False]) \
             .groupby(['nconst', 'startYear']) \
             .cumcount() + 1
imdb.tail(10)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,rate_diff,last_refresh,RN
2505012,tt9916538,NaN,8.3,6,nm8678236,actress,Lala Karmela,1985,0,Lala Karmela (1985 - ),...,0,2019,0,123,Drama,Fiction,Kuambil Lagi Hatiku (2019),NaN,20220826074833,1
2505013,tt9916730,NaN,8.4,6,nm0059461,"actor,music_department",Sunil Barve,0,0,Sunil Barve,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505014,tt9916730,NaN,8.4,6,nm10538612,NaN,Kiran Gawade,0,0,Kiran Gawade,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505015,tt9916730,NaN,8.4,6,nm10538613,producer,Abhishek Jathar,0,0,Abhishek Jathar,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505016,tt9916730,NaN,8.4,6,nm10538614,producer,Ujjwala Gawde,0,0,Ujjwala Gawde,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505017,tt9916730,NaN,8.4,6,nm13233318,"production_manager,actor",Ganesh Vasant Patil,0,0,Ganesh Vasant Patil,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505018,tt9916730,NaN,8.4,6,nm1957275,"cinematographer,camera_department,producer",Suresh Deshmane,0,0,Suresh Deshmane,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505019,tt9916730,NaN,8.4,6,nm4852679,actor,Bhushan Pradhan,0,0,Bhushan Pradhan,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505020,tt9916730,NaN,8.4,6,nm6096005,actor,Devadhar Archit,0,0,Devadhar Archit,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1
2505021,tt9916730,NaN,8.4,6,nm9785908,"assistant_director,editor",Rohita More,0,0,Rohita More,...,0,2017,0,116,\N,Fiction,6 Gunn (2017),NaN,20220826074833,1


In [11]:
#################################################
#remove rows not needed 
#################################################

X = imdb[imdb['category']!= "production_designer"] 
X = X[X['category']!= "archive_sound"] 
#X = X[X['category']!= "archive_footage"]
print(X.shape, imdb.shape)

#X.sort_values(by=['tconst', 'averageRating'], ascending=False)
#################################################
# export to file
#################################################
X.to_csv('data/imdb.csv'  ,encoding='utf-8')

(2478652, 28) (2505022, 28)


In [12]:
X[X['tconst'] == "tt1745960"].head(15)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,isAdult,startYear,endYear,runtimeMinutes,genres,flg_doc,primaryTitle_year,rate_diff,last_refresh,RN
2565,tt1745960,7.0,8.5,296003,nm0000124,actress,Jennifer Connelly,1970,0,Jennifer Connelly (1970 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,1
2566,tt1745960,7.0,8.5,296003,nm0000129,"actor,producer,soundtrack",Tom Cruise,1962,0,Tom Cruise (1962 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,1
2567,tt1745960,7.0,8.5,296003,nm0000174,"actor,soundtrack,producer",Val Kilmer,1959,0,Val Kilmer (1959 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,2
2568,tt1745960,7.0,8.5,296003,nm0143596,writer,Jim Cash,1941,2000,Jim Cash (1941 - 2000),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,1
2569,tt1745960,7.0,8.5,296003,nm0185976,"writer,producer,director",Peter Craig,1969,0,Peter Craig (1969 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,2
2570,tt1745960,7.0,8.5,296003,nm0258390,"writer,camera_department,producer",Jack Epps Jr.,1949,0,Jack Epps Jr. (1949 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,1
2571,tt1745960,7.0,8.5,296003,nm0472567,"writer,producer,executive",Ehren Kruger,1972,0,Ehren Kruger (1972 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,1
2572,tt1745960,7.0,8.5,296003,nm1098479,"writer,producer,miscellaneous",Justin Marks,0,0,Justin Marks,...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,1
2573,tt1745960,7.0,8.5,296003,nm1886602,"actor,producer,soundtrack",Miles Teller,1987,0,Miles Teller (1987 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,2
2574,tt1745960,7.0,8.5,296003,nm2676052,"director,writer,producer",Joseph Kosinski,1974,0,Joseph Kosinski (1974 - ),...,0,2022,0,130,"Action,Drama",Fiction,Top Gun: Maverick (2022),-1.5,20220826074833,2


In [13]:
#################################################
# Distinct NCONST aggregation
#################################################
import numpy as np
N_avg = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.mean]                   
                   , values = ["tconst_imdb_rating", "tconst_your_rating"]
      ##             , fill_value = 0
             )
N_sum = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.sum]                   
                   , values = [ "tconst_imdb_num_votes"]
                   , fill_value = 0
             )
N_max = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ np.max]                   
                   , values = [ "startYear"]
                   , fill_value = 0
             )
N_cnt = X.pivot_table(  index   = ( 'nconst', 'primaryName_year', 'cat_cln')
                   , aggfunc = [ 'count']  
                   , values = ["tconst"]
                   , fill_value = 0
             )

N_unique = pd.merge(   N_sum
                     , N_avg 
                     , how = 'inner', left_on=  [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_cnt
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )
N_unique = pd.merge(   N_unique
                     , N_max
                     , how = 'inner', left_on= [ 'nconst', 'primaryName_year', 'cat_cln'], right_on= [ 'nconst', 'primaryName_year', 'cat_cln']
                 )

#format column names - grouping created by pivo table
N_unique.columns = ['_'.join(str(s).strip() for s in col if s) for col in N_unique.columns]

#reset index
N_unique.reset_index(inplace=True)

N_unique.rename(columns = {  "sum_tconst_imdb_num_votes": "tconst_imdb_num_votes"
                           , "mean_tconst_imdb_rating":   "tconst_imdb_rating"
                           , "mean_tconst_your_rating":   "tconst_your_rating"
                           , "count_tconst":              "cnt_tconst"
                           , "amax_startYear":            "max_startYear"}
                           , inplace = True)

In [14]:
N_unique.head(3)

,nconst,primaryName_year,cat_cln,tconst_imdb_num_votes,tconst_imdb_rating,tconst_your_rating,cnt_tconst,max_startYear
0,nm0000001,Fred Astaire (1899 - 1987),actor,198399,6.717143,NaN,35,1981
1,nm0000001,Fred Astaire (1899 - 1987),self,9156,7.016667,NaN,12,2021
2,nm0000002,Lauren Bacall (1924 - 2014),actor,567417,6.400000,5.0,37,2012


In [15]:
# lst = ['nm0000001', 'nm0000002']
# N_unique =N_unique.query('nconst in @lst')
N_unique.to_csv('data/nconst.csv' , header=0 , encoding='utf-8')

In [16]:
############################################ 
### import festival inforamaion
############################################ 

var_list = ('Const','Title' , 'Description', 'Directors' , 'Genres', 'Release Date', 'URL')
 
############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls560308483/    Cannes 2022: Official Selections
## https://www.imdb.com/list/ls560342137     Cannes 75

fst = pd.DataFrame()
fst = pd.read_csv ("data/Cannes 2022 Official Selections.csv", encoding='latin-1')
fst = fst.loc[:, var_list]
fst.rename(columns = {    "Const": "tconst"}
                      , inplace = True)

fst['festival_name'] = 'Cannes'
fst['festival_year'] = '2022'

############################################ 
### import festival inforamaion - Sundance
############################################ 

## https://www.imdb.com/list/ls512369774/   sundance 2022
fst3 = pd.DataFrame()
fst3 = pd.read_csv ("data/Sundance 2022 Movie Guide.csv", encoding='latin-1')
fst3 = fst3.loc[:, var_list]
fst3.rename(columns = {    "Const": "tconst"} 
                      , inplace = True)
fst3['festival_name'] = 'Sundance'
fst3['festival_year'] = '2022'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls534810559/   sxsw 2022
fst4 = pd.DataFrame()
fst4 = pd.read_csv ("data/SXSW 2022 Festival Guide.csv", encoding='latin-1')
fst4 = fst4.loc[:, var_list]
fst4.rename(columns = {    "Const": "tconst"} 
                      , inplace = True)
fst4['festival_name'] = 'SXSW'
fst4['festival_year'] = '2022'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls565715845   Yesh 2022
fst1 = pd.DataFrame()
fst1 = pd.read_csv ("data/Yesh2022.csv", encoding='latin-1')
fst1 = fst1.loc[:, var_list]
fst1.rename(columns = {    "Const": "tconst"} 
                      , inplace = True)
fst1['festival_name'] = 'Yesh!'
fst1['festival_year'] = '2022'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls565715845   Yesh 2022
fst2 = pd.DataFrame()
fst2 = pd.read_csv ("data/Zurich Film Festival 2021.csv", encoding='latin-1')
fst2 = fst2.loc[:, var_list]
fst2.rename(columns = {   "Const": "tconst"}
                      , inplace = True)
fst2['festival_name'] = 'ZFF'
fst2['festival_year'] = '2021'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls567162891/   lff
fst5 = pd.DataFrame()
fst5 = pd.read_csv ("data/Locarno Film Festival 2022.csv", encoding='latin-1')
fst5 = fst5.loc[:, var_list]
fst5.rename(columns = {   "Const": "tconst"}
                      , inplace = True)
fst5['festival_name'] = 'LFF'
fst5['festival_year'] = '2022'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls567227005/  tiff
fst6 = pd.DataFrame()
fst6 = pd.read_csv ("data/2022 Toronto International Film Festival Movie Gui.csv", encoding='latin-1')
fst6 = fst6.loc[:, var_list]
fst6.rename(columns = {   "Const": "tconst"}
                      , inplace = True)
fst6['festival_name'] = 'TIFF'
fst6['festival_year'] = '2022'

############################################ 
### import festival inforamaion
############################################ 

## https://www.imdb.com/list/ls567153978/   what's on
fstw = pd.DataFrame()
fstw = pd.read_csv ("data/whatson.csv", encoding='latin-1')
fstw = fstw.loc[:, var_list]
fstw.rename(columns = {   "Const": "tconst"}
                      , inplace = True)
fstw['festival_name'] = 'WhatsOn'
fstw['festival_year'] = '2022'

union = pd.concat([fst, fst1])
union = pd.concat([union, fst2, fst3, fst4, fstw, fst5, fst6]) 
union.tail(3)

,tconst,Title,Description,Directors,Genres,Release Date,URL,festival_name,festival_year
130,tt21651682,Puerta a Puerta,"Festival Category: Wavelengths, Shorts","Jessica Sarah Rinland, Luis ArnÃ­as",Short,2022-09-30,https://www.imdb.com/title/tt21651682/,TIFF,2022
131,tt21651684,The Time That Separates Us,"Festival Category: Wavelengths, Shorts",Parastoo Anoushahpour,Short,2022-09-30,https://www.imdb.com/title/tt21651684/,TIFF,2022
132,tt21651690,What Rules The Invisible,"Festival Category: Wavelengths, Shorts",Tiffany Sia,Short,2022-09-30,https://www.imdb.com/title/tt21651690/,TIFF,2022


In [17]:
          
Xfst = pd.merge(    X
                  , union 
                  , how = 'inner', left_on="tconst", right_on="tconst"
                 )
Xfst.to_csv('data/Xfst.csv' , header=1 , encoding='utf-8')
Xfst.head(3)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,last_refresh,RN,Title,Description,Directors,Genres,Release Date,URL,festival_name,festival_year
0,tt10262648,8.0,7.3,9154,nm0041775,"actress,director",Yuliya Aug,1970,0,Yuliya Aug (1970 - ),...,20220826074833,3,Hytti nro 6,NaN,Juho Kuosmanen,Drama,2021-07-10,https://www.imdb.com/title/tt10262648/,ZFF,2021
1,tt10262648,8.0,7.3,9154,nm0510162,"writer,director,actress",Rosa Liksom,1958,0,Rosa Liksom (1958 - ),...,20220826074833,1,Hytti nro 6,NaN,Juho Kuosmanen,Drama,2021-07-10,https://www.imdb.com/title/tt10262648/,ZFF,2021
2,tt10262648,8.0,7.3,9154,nm0631792,"actor,director,writer",Valeriy Nikolaev,1965,0,Valeriy Nikolaev (1965 - ),...,20220826074833,1,Hytti nro 6,NaN,Juho Kuosmanen,Drama,2021-07-10,https://www.imdb.com/title/tt10262648/,ZFF,2021


In [18]:

union[union['festival_name'] == "LFF"].head(15)  

,tconst,Title,Description,Directors,Genres,Release Date,URL,festival_name,festival_year
0,tt21226542,De noche los gatos son pardos,NaN,Valentin Merz,Fantasy,2022-08-31,https://www.imdb.com/title/tt21226542/,LFF,2022
1,tt14189076,Ariyippu,NaN,Mahesh Narayanan,Drama,2022-08-03,https://www.imdb.com/title/tt14189076/,LFF,2022
2,tt21227108,Skazka,NaN,Aleksandr Sokurov,NaN,2022-08-31,https://www.imdb.com/title/tt21227108/,LFF,2022
3,tt13846508,Human Flowers of Flesh,NaN,Helena Wittmann,Drama,2022-08-31,https://www.imdb.com/title/tt13846508/,LFF,2022
4,tt14661418,Il pataffio,NaN,Francesco Lagi,Drama,2022-08-18,https://www.imdb.com/title/tt14661418/,LFF,2022
5,tt21149114,Matter Out of Place,NaN,Nikolaus Geyrhalter,Documentary,2022-08-31,https://www.imdb.com/title/tt21149114/,LFF,2022
6,tt21226876,Piaffe,NaN,Ann Oren,NaN,2022-08-31,https://www.imdb.com/title/tt21226876/,LFF,2022
7,tt15520368,Regra 34,NaN,JÃºlia Murat,Drama,2022-08-31,https://www.imdb.com/title/tt15520368/,LFF,2022
8,tt15012246,Bowling Saturne,NaN,Patricia Mazuy,"Drama, Thriller",2022-08-31,https://www.imdb.com/title/tt15012246/,LFF,2022
9,tt21226152,Sermon to the Fish,NaN,Hilal Baydarov,NaN,2022-08-31,https://www.imdb.com/title/tt21226152/,LFF,2022


In [19]:

# duplicate for rating dimension

import numpy as np
# Xfst2 = pd.DataFrame({'Xfst':['tconst','nconst'],})
# Xfst2 = pd.DataFrame({'Xfst':['tconst','nconst'],})
Xfst2 = Xfst.copy()   # panda refernces tables only, so explicit copy required
  
Xfst2['rating_category']           = 'IMDb'
Xfst2['tconst_num_votes']          = Xfst2['tconst_imdb_num_votes']   
Xfst2['tconst_rating']             = Xfst2['tconst_imdb_rating']   
Xfst2.head(3)

Xfst3 = Xfst.copy()
  
Xfst3['rating_category']           = 'Profile_1'
#Xfst3['tconst_num_votes']          = 1   
#Xfst3['tconst_rating']             = np.where(Xfst3['tconst_your_rating'].isnull(), 0, Xfst3['tconst_your_rating'] )
Xfst3['tconst_num_votes']       = np.where(Xfst3['tconst_your_rating'].isnull(), 0, 1)
Xfst3['tconst_rating']          = np.where(Xfst3['tconst_your_rating'].isnull(), np.nan, Xfst3['tconst_your_rating'] )
Xfst3.head(3)
 

 
rating_duplicate = pd.concat([Xfst2, Xfst3], ignore_index=True) 
rating_duplicate.to_csv('data/rating_duplicate.csv' , header=1 , encoding='utf-8')

In [20]:
rating_duplicate.tail(4)

,tconst,tconst_your_rating,tconst_imdb_rating,tconst_imdb_num_votes,nconst,primaryProfession,primaryName,birthYear,deathYear,primaryName_year,...,Description,Directors,Genres,Release Date,URL,festival_name,festival_year,rating_category,tconst_num_votes,tconst_rating
9234,tt9883832,NaN,7.8,143,nm0000309,"soundtrack,composer,actor",David Bowie,1947,2016,David Bowie (1947 - 2016),...,Category: MIDNIGHT SCREENINGS,Brett Morgen,"Documentary, Music",2022-05-23,https://www.imdb.com/title/tt9883832/,Cannes,2022,Profile_1,0,NaN
9235,tt9883832,NaN,7.8,143,nm0000309,"soundtrack,composer,actor",David Bowie,1947,2016,David Bowie (1947 - 2016),...,Festival Category: Special Presentations,Brett Morgen,"Documentary, Music",2022-05-23,https://www.imdb.com/title/tt9883832/,TIFF,2022,Profile_1,0,NaN
9236,tt9883832,NaN,7.8,143,nm0605137,"producer,director,writer",Brett Morgen,1968,0,Brett Morgen (1968 - ),...,Category: MIDNIGHT SCREENINGS,Brett Morgen,"Documentary, Music",2022-05-23,https://www.imdb.com/title/tt9883832/,Cannes,2022,Profile_1,0,NaN
9237,tt9883832,NaN,7.8,143,nm0605137,"producer,director,writer",Brett Morgen,1968,0,Brett Morgen (1968 - ),...,Festival Category: Special Presentations,Brett Morgen,"Documentary, Music",2022-05-23,https://www.imdb.com/title/tt9883832/,TIFF,2022,Profile_1,0,NaN


In [21]:
N_unique.head(3)

,nconst,primaryName_year,cat_cln,tconst_imdb_num_votes,tconst_imdb_rating,tconst_your_rating,cnt_tconst,max_startYear
0,nm0000001,Fred Astaire (1899 - 1987),actor,198399,6.717143,NaN,35,1981
1,nm0000001,Fred Astaire (1899 - 1987),self,9156,7.016667,NaN,12,2021
2,nm0000002,Lauren Bacall (1924 - 2014),actor,567417,6.400000,5.0,37,2012


In [22]:
# duplicate for rating dimension



N2 = N_unique.copy()   # panda refernces tables only, so explicit copy required
  
N2['rating_category']           = 'IMDb'
N2['tconst_num_votes']          = N2['tconst_imdb_num_votes']   
N2['tconst_rating']             = N2['tconst_imdb_rating']   
N2.head(3)

N3 = N_unique.copy()
  
N3['rating_category']           = 'Profile_1'
N3['tconst_num_votes']          = np.where(N3['tconst_your_rating'].isnull(), 0, 1 )
N3['tconst_rating']             = np.where(N3['tconst_your_rating'].isnull(), np.nan, N3['tconst_your_rating'] )
N3.head(3)
 


nconst_duplicate = pd.concat([N2, N3], ignore_index=True) 
nconst_duplicate.to_csv('data/nconst_duplicate.csv' , header=1 , encoding='utf-8')


In [23]:
print(fst.shape, Xfst.shape)
nconst_duplicate.head(3)

(66, 9) (4619, 36)


,nconst,primaryName_year,cat_cln,tconst_imdb_num_votes,tconst_imdb_rating,tconst_your_rating,cnt_tconst,max_startYear,rating_category,tconst_num_votes,tconst_rating
0,nm0000001,Fred Astaire (1899 - 1987),actor,198399,6.717143,NaN,35,1981,IMDb,198399,6.717143
1,nm0000001,Fred Astaire (1899 - 1987),self,9156,7.016667,NaN,12,2021,IMDb,9156,7.016667
2,nm0000002,Lauren Bacall (1924 - 2014),actor,567417,6.400000,5.0,37,2012,IMDb,567417,6.400000


In [24]:
Xfst.tail(3)
P[P['tconst'] == "tt14189076"].head(15)

,tconst,ordering,nconst,category,job,characters,cat_cln
19995276,tt14189076,1,nm0090283,actor,\N,\N,actor
19995277,tt14189076,2,nm6763126,actor,\N,\N,actor
19995278,tt14189076,3,nm2240830,actor,\N,\N,actor
19995279,tt14189076,4,nm8407664,actor,\N,\N,actor
19995280,tt14189076,5,nm3525879,director,\N,\N,director
19995281,tt14189076,6,nm3616989,producer,producer,\N,producer
19995282,tt14189076,7,nm5444706,composer,\N,\N,composer
19995283,tt14189076,8,nm2062094,cinematographer,\N,\N,cinematographer
19995284,tt14189076,9,nm11817314,editor,\N,\N,editor
